In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv("nitems.csv")

In [8]:
df.head()

,order_id,user_id,eval_set,order_dow,order_hour_of_day,days_since_prior_order,n_basket_items,avg_basket,hour_day_bucket,avg_time_bucket_basket_0,...,fav_dpt_14,fav_dpt_15,fav_dpt_16,fav_dpt_17,fav_dpt_18,fav_dpt_19,fav_dpt_20,fav_dpt_21,n_last_order,n_highest
0,1187899,1,train,4,8,14,10,5.185714,2,4.230769,...,0,0,0,0,0,1,0,0,4,10
1,1492625,2,train,1,11,30,12,8.646018,2,0.000000,...,0,0,1,0,0,0,0,0,11,14
2,2196797,5,train,0,11,6,4,3.869565,2,3.785714,...,0,0,0,0,0,0,0,0,2,8
3,"525,192",7,train,2,11,6,8,8.316279,1,4.795918,...,0,0,0,0,0,0,0,0,4,20
4,"880,375",8,train,1,14,10,4,3.835821,1,1.911765,...,0,0,0,0,0,0,0,0,5,8


In [34]:
np.random.seed(7)
msk = np.random.rand(len(df)) < 0.8
y_col = ['n_basket_items']
x_col = ['order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'avg_basket',
       'hour_day_bucket', 'avg_time_bucket_basket_0',
       'avg_time_bucket_basket_1', 'avg_time_bucket_basket_2',
       'avg_time_bucket_basket_3', 'department_id', 'fav_dpt_1', 'fav_dpt_2',
       'fav_dpt_3', 'fav_dpt_4', 'fav_dpt_5', 'fav_dpt_6', 'fav_dpt_7',
       'fav_dpt_8', 'fav_dpt_9', 'fav_dpt_10', 'fav_dpt_11', 'fav_dpt_12',
       'fav_dpt_13', 'fav_dpt_14', 'fav_dpt_15', 'fav_dpt_16', 'fav_dpt_17',
       'fav_dpt_18', 'fav_dpt_19', 'fav_dpt_20', 'fav_dpt_21', 'n_last_order',
       'n_highest']

train = df[msk]
test = df[~msk]
train_y, train_x = train[y_col], train[x_col]
test_y, test_x = test[y_col], test[x_col]

In [35]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


print("Fit regression model")
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(train_x, train_y)
regr_2.fit(train_x, train_y)

print("Predict")
y_1 = regr_1.predict(test_x)
y_2 = regr_2.predict(test_x)



'''
# Plot the results
plt.figure()
plt.scatter(X, y, c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()'''

Fit regression model
Predict


'\n# Plot the results\nplt.figure()\nplt.scatter(X, y, c="darkorange", label="data")\nplt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)\nplt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)\nplt.xlabel("data")\nplt.ylabel("target")\nplt.title("Decision Tree Regression")\nplt.legend()\nplt.show()'

In [38]:
mse1 = mean_squared_error(y_1, test_y.as_matrix())
print(mse1)
mse2 = np.sqrt(mean_squared_error(y_2, test_y.as_matrix()))
print(mse2)

15.2477684629
3.54697698596


In [44]:
#now for adaboost
from sklearn.ensemble import AdaBoostRegressor

rng = np.random.RandomState(7)
ada_model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=rng)


ada_model.fit(train_x, train_y.values.ravel())

# Predict
ada_y = ada_model.predict(test_x)

In [45]:
mse_ada = mean_squared_error(ada_y, test_y.as_matrix())
mse_ada

15.976303428141012

In [52]:
score(test_x, test_yy, sample_weight=None)

NameError: name 'score' is not defined

In [75]:
'''
The real deal. Setting up grid search methods for fine tuning GBR
'''
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor  #GBM algorithm
from sklearn.tree import DecisionTreeRegressor #decsion tree regression
from sklearn.ensemble import AdaBoostRegressor #AdaBoost
from sklearn import metrics
from sklearn.grid_search import GridSearchCV   #for grid search over parameters
import matplotlib.pyplot as plt


In [2]:
import pandas as pd
dept_df=pd.read_csv('user_department_predictions.csv')
dept_df.head()

,user_id,dept,dept_id,prob_purchased
0,2,meat seafood,12,0.120269
1,7,meat seafood,12,0.345530
2,9,meat seafood,12,0.286382
3,10,meat seafood,12,0.634088
4,14,meat seafood,12,0.168476


In [68]:
#read department csv
dept_df=pd.read_csv('user_department_predictions.csv')

#turn these into vectors
def category_vectors(df):
    #make a df of just user ids
    user_dept_df = pd.DataFrame()
    user_dept_df['user_id'] = df.user_id.unique()

    for dept_n in range(1,22):
        #create key
        key = "dept_score_"+str(dept_n)
        
        temp_df = pd.DataFrame()
        temp_df[['user_id',key]] = df[df.dept_id == dept_n][['user_id','prob_purchased']]

        user_dept_df = pd.merge(user_dept_df, temp_df, on='user_id', how="left")
        
    return user_dept_df

user_dept_df = category_vectors(df)
user_dept_df.fillna(0, inplace=True)
user_dept_df.head()

,user_id,dept_score_1,dept_score_2,dept_score_3,dept_score_4,dept_score_5,dept_score_6,dept_score_7,dept_score_8,dept_score_9,...,dept_score_12,dept_score_13,dept_score_14,dept_score_15,dept_score_16,dept_score_17,dept_score_18,dept_score_19,dept_score_20,dept_score_21
0,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.343027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.286242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.634655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.169770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.201918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
#read the main and merge wityh department scores
n_items_df = pd.read_csv('n_items.csv')
n_items_df = pd.merge(n_items_df, user_dept_df, on="user_id", how="left")
n_items_df.fillna(0, inplace=True)

#split it and never use the test set please...
np.random.seed(7)
msk = np.random.rand(len(n_items_df)) < 0.9
train = n_items_df[msk]
test = n_items_df[~msk]

#columns
y_col = ['n_basket_items']
x_col = ['order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'avg_basket',
       'avg_time_bucket_basket_0', 'avg_time_bucket_basket_1',
       'avg_time_bucket_basket_2', 'avg_time_bucket_basket_3', 'n_last_order',
       'n_highest', 'std_basket', 'dept_score_1', 'dept_score_2',
       'dept_score_3', 'dept_score_4', 'dept_score_5', 'dept_score_6',
       'dept_score_7', 'dept_score_8', 'dept_score_9', 'dept_score_10',
       'dept_score_11', 'dept_score_12', 'dept_score_13', 'dept_score_14',
       'dept_score_15', 'dept_score_16', 'dept_score_17', 'dept_score_18',
       'dept_score_19', 'dept_score_20', 'dept_score_21']

In [79]:
#cross validation function
def cv_modelfit(alg, dtrain, y, predictors, performCV=False, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    print("fit")
    alg.fit(dtrain[predictors], dtrain[y].values.ravel())
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    
    #Perform cross-validation:

    if performCV:
        print("CV")
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[y].values.ravel(), cv=cv_folds, scoring='neg_mean_squared_error')
 
    #Print model report:
    print ("\nModel Report")
    print ("MSE : %.4g" % metrics.mean_squared_error(dtrain[y].values, dtrain_predictions))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')

In [83]:
#baseline models:
rng = np.random.RandomState(7)

gbm0 = GradientBoostingClassifier(random_state=rng)
dtr = DecisionTreeRegressor(max_depth=5)
ada = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),
                          n_estimators=60, random_state=rng)

print("DTR")
cv_modelfit(dtr, train, y_col, x_col)
print("ADA")
cv_modelfit(ada, train, y_col, x_col)

DTR
fit
CV

Model Report
MSE : 12.22
CV Score : Mean - -12.4303 | Std - 0.166633 | Min - -12.71544 | Max - -12.23039
ADA
fit


/Users/austin.slakey/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CV


/Users/austin.slakey/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/austin.slakey/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/austin.slakey/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/austin.slakey/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array w


Model Report
MSE : 12.92
CV Score : Mean - -13.35862 | Std - 0.3912215 | Min - -13.85166 | Max - -12.67697
